In [1]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

#The purpose of this notebook is to test the hyperparameters found during the literature review for a similar task, on our preventative maintenance dataset.

# 1. Load Data
X_train = pd.read_csv('../data/processed/X_train_scaled.csv')
X_val   = pd.read_csv('../data/processed/X_val_scaled.csv')

y_train_df = pd.read_csv('../data/processed/Y_train.csv')
y_val_df   = pd.read_csv('../data/processed/Y_val.csv')

# Remove binary "Machine failure"
y_train_df = y_train_df.drop(columns=["Machine failure"])
y_val_df   = y_val_df.drop(columns=["Machine failure"])

# 2. Convert 1-hot failure indicators → multiclass target
failure_cols = ["TWF", "HDF", "PWF", "OSF", "RNF"]

def make_multiclass(df):
    df = df.copy()
    target = pd.Series(0, index=df.index)  # 0 = no failure
    for i, col in enumerate(failure_cols, start=1):
        target[df[col] == 1] = i
    return target

y_train = make_multiclass(y_train_df)
y_val   = make_multiclass(y_val_df)

# 3. Compute class weights (important for imbalance)
classes = np.unique(y_train)

cw = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, cw))

print("Class Weights:", class_weights)

# 4. Build the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(16, activation='relu'),

    Dense(6, activation='softmax')   # 6 classes
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 5. Train (unchanged parameters)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=40,
    batch_size=64,
    class_weight=class_weights,
    verbose=1
)

# 6. Evaluation
loss, acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\nValidation Accuracy: {acc:.4f}")
print(f"Validation Loss:     {loss:.4f}")

# Predictions
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report:")
print(classification_report(y_val, y_pred, digits=4))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Class Weights: {0: 0.17250977685760294, 1: 39.057291666666664, 2: 16.023504273504273, 3: 19.528645833333332, 4: 18.93686868686869, 5: 89.27380952380952}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,158 (12.34 KB)

 Trainable params: 3,158 (12.34 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1909 - loss: 1.8584 - val_accuracy: 0.3167 - val_loss: 1.7696
Epoch 2/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.2445 - loss: 1.6565 - val_accuracy: 0.3180 - val_loss: 1.7200
Epoch 3/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.3336 - loss: 1.4348 - val_accuracy: 0.4713 - val_loss: 1.4589
Epoch 4/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.4116 - loss: 1.2434 - val_accuracy: 0.4313 - val_loss: 1.3937
Epoch 5/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.4406 - loss: 1.0873 - val_accuracy: 0.5067 - val_loss: 1.2501
Epoch 6/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.4877 - loss: 1.0295 - val_accuracy: 0.4747 - val_loss: 1.1770
Epoch 7/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - accuracy: 0.4096 - loss: 0.9969 - val_accuracy: 0.4447 - val_loss: 1.2453
Epoch 8/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - accuracy: 0.4752 - loss: 0.8710 - val_accuracy: 0.47